# covid dataset sep 2020 -  WHOIS analysis

In [1]:
import json
import pandas as pd
from collections import Counter
from collections import defaultdict as ddict
from matplotlib import pyplot as plt
import seaborn as sns
import tldextract

In [2]:
domain_scan_whois_pahth = '/export/sec02/nabeel/ucsc/Vaccine_Disinfo/data/covid_resources/oxford-vaccine-article-domains-shared-on-twitter.csv.whois'
domain_scan_whois = list(map(json.loads,open(domain_scan_whois_pahth,'r').readlines()))

In [3]:
def transform(x):
    for y in x:
        for i in y.get('data'):
            yield {
            'Domain Name':  i.get('attributes').get('whois_map').get('Domain Name'),
            'Creation Date':i.get('attributes').get('whois_map').get('Creation Date'),
            'expires':      i.get('attributes').get('whois_map').get('expires'),
            'Changed':      i.get('attributes').get('whois_map').get('Changed'),
            'Name Server':  i.get('attributes').get('whois_map').get('Name Server'),
            'nic-hdl-br':   i.get('attributes').get('whois_map').get('nic-hdl-br'),
            'status':       i.get('attributes').get('whois_map').get('status'),
            'registrant_country':i.get('attributes').get('registrant_country')
            }

In [4]:
df = pd.DataFrame(list(transform(domain_scan_whois)))

In [5]:
df.head(5)

,Domain Name,Creation Date,expires,Changed,Name Server,nic-hdl-br,status,registrant_country
0,None,None,None,None,None,None,None,None
1,forbes.com.br,19991201 #227065 | 20020619,20201201,20181011 | 20191031,leah.ns.cloudflare.com | stan.ns.cloudflare.com,NAB51,published,None
2,forbes.com.br,19991201 #227065 | 20020619,20201201,20181011 | 20191031,leah.ns.cloudflare.com | stan.ns.cloudflare.com,NAB51,published,BR
3,forbes.com.br,19991201 #227065 | 20020619,20191201,20181011 | 20181030,becky.ns.cloudflare.com | elliot.ns.cloudflare...,NAB51,published,BR
4,forbes.com.br,19991201 #227065 | 20020619,20191201,20181011 | 20181030,becky.ns.cloudflare.com | elliot.ns.cloudflare...,NAB51,published,None


In [6]:
NS = set()
NS_count = list()
for i in df['Name Server'].unique():
    if i:
        data = i.split('|')
        for y in data:
            ns = tldextract.extract(y)
            NS.add('{}.{}'.format(ns.domain,ns.suffix))
            NS_count.append('{}.{}'.format(ns.domain,ns.suffix))            

In [7]:
print("Number of unique NS {}".format(len(NS)))

Number of unique NS 2521


In [8]:
NSDF=ddict(list)
for ns, count in list(sorted(Counter(NS_count).items(), key=lambda x: x[1],reverse=True)):
                   NSDF['NS'].append(ns)
                   NSDF['count'].append(count)
NSDF = pd.DataFrame(NSDF)

In [9]:
NSDF.head(10)

,NS,count
0,CLOUDFLARE.COM,1681
1,cloudflare.com,1062
2,AKAM.NET,409
3,akam.net,239
4,NSONE.NET,214
5,DYNECT.NET,200
6,nsone.net,149
7,DOMAINCONTROL.COM,138
8,dynect.net,131
9,domaincontrol.com,102


In [10]:
NSDF.tail(10)

,NS,count
2511,radarcirebon.net,1
2512,AURORADNS.EU,1
2513,AURORADNS.NL,1
2514,AURORADNS.INFO,1
2515,auroradns.eu,1
2516,auroradns.nl,1
2517,auroradns.info,1
2518,ignum.cz,1
2519,"J,_�^u .",1
2520,�� ��K�o� 8\t�.,1


In [11]:
df.groupby(['Domain Name','Name Server']).size()

Domain Name           Name Server                                                                                                                                  
100MILEFREEPRESS.NET  PDNS03.DOMAINCONTROL.COM | PDNS04.DOMAINCONTROL.COM                                                                                              3
100e7.com.br          ns1.dreamhost.com | ns2.dreamhost.com | ns3.dreamhost.com                                                                                        3
102NUEVE.COM          NS1.BLUEHOST.COM | NS2.BLUEHOST.COM | ns1.bluehost.com | ns2.bluehost.com                                                                        1
                      NS1MPZ.NAME.COM | NS2GTX.NAME.COM | NS3FLT.NAME.COM | NS4BHT.NAME.COM | ns1mpz.name.com | ns2gtx.name.com | ns3flt.name.com | ns4bht.name.com    1
1053RNB.COM           NS1.WORDPRESS.COM | NS2.WORDPRESS.COM                                                                                                     